In [1]:
import numpy as np
import pandas as pd
import random
import warnings
warnings.filterwarnings("ignore")

In [2]:
stock = np.array([22.0, 3.7, 4.5, 47.0, 18.5, 5.0])
demand = np.array([3.0, 2.6, 16.0, 20.0, 26.4, 11.9])
cost = np.matrix('92.63, 93.25, 112.31, 73.34, 89.15, 149.24')
duty = np.matrix('0, 0, 0.5, 0.095, 0.045, 0.06; 0.6, 0, 0.5, 0.095, 0.045, 0.06;\
0.6, 0, 0, 0.095, 0.045, 0.06; 0.6, 0, 0.5, 0, 0.045, 0.06;\
0.6, 0, 0.5, 0.095, 0, 0.06; 0.6, 0, 0.5, 0.095, 0.045, 0')
shipping = np.matrix('0 11.4 7 11 11 14; 11 0 9 11.5 6 13; 7 10 0 13 10.4 14.3;\
10 11.5 12.5 0 11.2 13.3; 10 6 11 10 0 12.5; 14 13 12.5 14.2 13 0')
plan = np.matrix('3 0 0 0 12.4 1.8;0 2.6 0 0 0 0; 0 0 4.1 0 0 0; 0 0 11.9 20 0 6.1;\
0 0 0 0 14 0; 0 0 0 0 0 4')

In [4]:
print("Stock:")
print(stock)

Stock:
[22.   3.7  4.5 47.  18.5  5. ]


In [5]:
print("Demand:")
print(demand)

Demand:
[ 3.   2.6 16.  20.  26.4 11.9]


In [6]:
c = np.multiply((1+duty), cost.T) + shipping
print("C matrix:")
print(c)

C matrix:
[[ 92.63    104.03    145.945   112.42985 107.79835 112.1878 ]
 [160.2      93.25    148.875   113.60875 103.44625 111.845  ]
 [186.696   122.31    112.31    135.97945 127.76395 133.3486 ]
 [127.344    84.84    122.51     73.34     87.8403   91.0404 ]
 [152.64     95.15    144.725   107.61925  89.15    106.999  ]
 [252.784   162.24    236.36    177.6178  168.9558  149.24   ]]


## Project Plan:

In [6]:
np.multiply(c,plan).sum()

7844.524020000001

In [7]:
#define penalty function for both row and column
#input: cost matrix, output: column penalty, row penalty
def penalty(c):
    #deep copy cost matrix
    col_rank = np.copy(c)
    row_rank = np.copy(c)
    #sort cost matrix by row and column separately
    col_rank.sort(axis=0)
    row_rank.sort(axis=1)
    #compute difference between lowest cost to the second lowest
    penalty_col = row_rank[:,1] - row_rank[:,0]
    penalty_row = col_rank[1,:] - col_rank[0,:]
    #change all nan value to zero
    penalty_col = np.nan_to_num(penalty_col)
    penalty_row = np.nan_to_num(penalty_row)
    #exlude penalty too large, happen when cost function mapped to infinity
    penalty_col[penalty_col > 1000] = 0
    penalty_row[penalty_row > 1000] = 0
    
    return penalty_col, penalty_row

In [8]:
#find the best transaction location based on penalty
def find_location(c, penalty_col, penalty_row):
    #row penalty larger than column, search for lowest cost in the column
    if np.max(penalty_row) > np.max(penalty_col):
        best_buyer = np.argmax(penalty_row)
        supply_list = c[:,best_buyer]
        best_supplier = np.argmin(supply_list)
    #column penalty larger than row, search for lowest cost in the row
    elif np.max(penalty_row) < np.max(penalty_col):
        best_supplier = np.argmax(penalty_col)
        demand_list = c[best_supplier,:]
        best_buyer = np.argmin(demand_list)
    #tie breaker, randomize row/column picked, not happened in this assignment
    elif np.max(penalty_row) == np.max(penalty_col) and random.choice([0,1]) == 0:
        best_buyer = np.argmax(penalty_row)
        supply_list = c[:,best_buyer]
        best_supplier = np.argmin(supply_list)
    elif np.max(penalty_row) == np.max(penalty_col) and random.choice([0,1]) == 1:
        best_supplier = np.argmax(penalty_col)
        demand_list = c[best_supplier,:]
        best_buyer = np.argmin(demand_list)
    return best_supplier, best_buyer

In [9]:
#complete transaction and update cost, demand, supply information, cache transaction history
def locate_resource(c, best_supplier, best_buyer, stock, demand, transaction):
    #more stock than demand, satisfy all demand
    if stock[best_supplier] > demand[best_buyer]:
        item = demand[best_buyer]
        c[:, best_buyer] = np.inf
    else:
    #more demand than stock, exhaust all stock
        item = stock[best_supplier]
        c[best_supplier, :] = np.inf
    #update all item in stock, demand, transaction
    stock[best_supplier] -= item
    demand[best_buyer] -= item
    transaction[best_supplier, best_buyer] += item
    return c, stock, demand, transaction

In [10]:
def vam(c, stock, demand):
    #copy all information, do not want to change original input
    c_copy = np.copy(c)
    stock_copy = np.copy(stock)
    demand_copy = np.copy(demand)
    #initialize transaction history    
    transaction = np.zeros((len(stock),len(demand)))
    print(stock_copy)
    print(demand_copy)
    #while loop as long as demand is not satisfied
    #if stock is smaller than demand, break from loop
    while np.max(demand_copy) > 0:
        if np.sum(stock) < np.sum(demand):
          print('Not enought resourses')
          break
        penalty_demand, penalty_supply = penalty(c_copy)
        best_supplier, best_buyer = find_location(c_copy, penalty_demand, penalty_supply)
        c_copy, stock_copy, demand_copy, transaction = locate_resource(c_copy,
                                                               best_supplier, best_buyer, 
                                                               stock_copy, demand_copy, transaction)
        if np.sort(stock_copy)[-2] == 0 and np.sort(demand_copy)[-2] == 0:
            item = np.max(demand_copy)
            best_buyer = np.argmax(demand_copy)
            best_supplier = np.argmax(stock_copy)
            stock_copy[best_supplier] -= item
            demand_copy[best_buyer] -= item
            transaction[best_supplier, best_buyer] += item
        print('Transaction Pair:')
        print(best_supplier, best_buyer)
        print(stock_copy)
        print(demand_copy)
    #print transation history
    print(transaction)
    total_cost = np.multiply(c,transaction).sum()
    return total_cost

In [11]:
total_cost = vam(c, stock, demand)
total_cost

[22.   3.7  4.5 47.  18.5  5. ]
[ 3.   2.6 16.  20.  26.4 11.9]
Transaction Pair:
0 0
[19.   3.7  4.5 47.  18.5  5. ]
[ 0.   2.6 16.  20.  26.4 11.9]
Transaction Pair:
3 3
[19.   3.7  4.5 27.  18.5  5. ]
[ 0.   2.6 16.   0.  26.4 11.9]
Transaction Pair:
3 5
[19.   3.7  4.5 15.1 18.5  5. ]
[ 0.   2.6 16.   0.  26.4  0. ]
Transaction Pair:
2 2
[19.   3.7  0.  15.1 18.5  5. ]
[ 0.   2.6 11.5  0.  26.4  0. ]
Transaction Pair:
3 2
[19.   3.7  0.   3.6 18.5  5. ]
[ 0.   2.6  0.   0.  26.4  0. ]
Transaction Pair:
1 1
[19.   1.1  0.   3.6 18.5  5. ]
[ 0.   0.   0.   0.  26.4  0. ]
Transaction Pair:
3 4
[19.   1.1  0.   0.  18.5  5. ]
[ 0.   0.   0.   0.  22.8  0. ]
Transaction Pair:
4 4
[19.   1.1  0.   0.   0.   5. ]
[0.  0.  0.  0.  4.3 0. ]
Transaction Pair:
1 4
[19.  0.  0.  0.  0.  5.]
[0.  0.  0.  0.  3.2 0. ]
Transaction Pair:
0 4
[15.8  0.   0.   0.   0.   5. ]
[0. 0. 0. 0. 0. 0.]
[[ 3.   0.   0.   0.   3.2  0. ]
 [ 0.   2.6  0.   0.   1.1  0. ]
 [ 0.   0.   4.5  0.   0.   0. ]
 [ 0.  

7409.026435

In [12]:
total_cost_85 = vam(c, stock*0.85, demand)
total_cost_85

[18.7    3.145  3.825 39.95  15.725  4.25 ]
[ 3.   2.6 16.  20.  26.4 11.9]
Transaction Pair:
0 0
[15.7    3.145  3.825 39.95  15.725  4.25 ]
[ 0.   2.6 16.  20.  26.4 11.9]
Transaction Pair:
3 3
[15.7    3.145  3.825 19.95  15.725  4.25 ]
[ 0.   2.6 16.   0.  26.4 11.9]
Transaction Pair:
3 5
[15.7    3.145  3.825  8.05  15.725  4.25 ]
[ 0.   2.6 16.   0.  26.4  0. ]
Transaction Pair:
2 2
[15.7    3.145  0.     8.05  15.725  4.25 ]
[ 0.     2.6   12.175  0.    26.4    0.   ]
Transaction Pair:
3 2
[15.7    3.145  0.     0.    15.725  4.25 ]
[ 0.     2.6    4.125  0.    26.4    0.   ]
Transaction Pair:
4 4
[15.7    3.145  0.     0.     0.     4.25 ]
[ 0.     2.6    4.125  0.    10.675  0.   ]
Transaction Pair:
1 1
[15.7    0.545  0.     0.     0.     4.25 ]
[ 0.     0.     4.125  0.    10.675  0.   ]
Transaction Pair:
5 4
[15.7    0.545  0.     0.     0.     0.   ]
[0.    0.    4.125 0.    6.425 0.   ]
Transaction Pair:
1 4
[15.7  0.   0.   0.   0.   0. ]
[0.    0.    4.125 0.    5.88  0

7898.51353925

In [13]:
stock_wo_chile = np.array([22.0, 3.7, 0, 47.0, 18.5, 5.0])
total_cost_wo_chile = vam(c, stock_wo_chile, demand)
total_cost_wo_chile

[22.   3.7  0.  47.  18.5  5. ]
[ 3.   2.6 16.  20.  26.4 11.9]
Transaction Pair:
0 0
[19.   3.7  0.  47.  18.5  5. ]
[ 0.   2.6 16.  20.  26.4 11.9]
Transaction Pair:
3 3
[19.   3.7  0.  27.  18.5  5. ]
[ 0.   2.6 16.   0.  26.4 11.9]
Transaction Pair:
3 5
[19.   3.7  0.  15.1 18.5  5. ]
[ 0.   2.6 16.   0.  26.4  0. ]
Transaction Pair:
2 2
[19.   3.7  0.  15.1 18.5  5. ]
[ 0.   2.6 16.   0.  26.4  0. ]
Transaction Pair:
3 2
[19.   3.7  0.   0.  18.5  5. ]
[ 0.   2.6  0.9  0.  26.4  0. ]
Transaction Pair:
4 4
[19.   3.7  0.   0.   0.   5. ]
[0.  2.6 0.9 0.  7.9 0. ]
Transaction Pair:
1 1
[19.   1.1  0.   0.   0.   5. ]
[0.  0.  0.9 0.  7.9 0. ]
Transaction Pair:
5 4
[19.   1.1  0.   0.   0.   0. ]
[0.  0.  0.9 0.  2.9 0. ]
Transaction Pair:
1 4
[19.  0.  0.  0.  0.  0.]
[0.  0.  0.9 0.  1.8 0. ]
Transaction Pair:
0 2
[16.3  0.   0.   0.   0.   0. ]
[0. 0. 0. 0. 0. 0.]
[[ 3.   0.   0.9  0.   1.8  0. ]
 [ 0.   2.6  0.   0.   1.1  0. ]
 [ 0.   0.   0.   0.   0.   0. ]
 [ 0.   0.  15.1 20

7853.654165

In [14]:
total_cost_85_wo_chile = vam(c, stock_wo_chile*0.85, demand)
total_cost_85_wo_chile

[18.7    3.145  0.    39.95  15.725  4.25 ]
[ 3.   2.6 16.  20.  26.4 11.9]
Transaction Pair:
0 0
[15.7    3.145  0.    39.95  15.725  4.25 ]
[ 0.   2.6 16.  20.  26.4 11.9]
Transaction Pair:
3 3
[15.7    3.145  0.    19.95  15.725  4.25 ]
[ 0.   2.6 16.   0.  26.4 11.9]
Transaction Pair:
3 5
[15.7    3.145  0.     8.05  15.725  4.25 ]
[ 0.   2.6 16.   0.  26.4  0. ]
Transaction Pair:
2 2
[15.7    3.145  0.     8.05  15.725  4.25 ]
[ 0.   2.6 16.   0.  26.4  0. ]
Transaction Pair:
3 2
[15.7    3.145  0.     0.    15.725  4.25 ]
[ 0.    2.6   7.95  0.   26.4   0.  ]
Transaction Pair:
4 4
[15.7    3.145  0.     0.     0.     4.25 ]
[ 0.     2.6    7.95   0.    10.675  0.   ]
Transaction Pair:
1 1
[15.7    0.545  0.     0.     0.     4.25 ]
[ 0.     0.     7.95   0.    10.675  0.   ]
Transaction Pair:
5 4
[15.7    0.545  0.     0.     0.     0.   ]
[0.    0.    7.95  0.    6.425 0.   ]
Transaction Pair:
1 4
[15.7  0.   0.   0.   0.   0. ]
[0.   0.   7.95 0.   5.88 0.  ]
Transaction Pair:


8027.16741425